In [1]:
import pymysql,sys
import seaborn as sns
import pandas as pd
from dateutil import parser
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
conn = pymysql.connect(host='localhost',user='root',passwd='pulsetini', db='nairaland',charset='utf8')
cur = conn.cursor()
cur2 = conn.cursor()

In [4]:
df=pd.read_sql_query('select * from comments_updated',conn)

In [5]:
df2=pd.read_sql_query('select * from posts3',conn)

In [8]:
df3=pd.read_sql_query('select * from comments3',conn)

### Take newly scraped comments from comments_updated, test to see if already present in comments to avoid duplicates. Then adjust comment numbering based on how many comments are in DB already

In [83]:
def addComment(c):
    q=u'insert into comments3 (parent,nComment,user,time,content,relevant,nLikes,nShares,isNew) values (%s,%s,%s,%s,%s,%s,%s,%s,%s)'
    cleanedLikes=c.nLikes
    if pd.isnull(cleanedLikes):
        print 'LIKES NULL',cleanedLikes
        cleanedLikes=0
    
    cleanedShares=c.nShares
    if pd.isnull(cleanedShares):
        cleanedShares=0

    data=[c.parent,-1,c.user,c.time.isoformat(),c.content,c.relevant,cleanedLikes,cleanedShares,True]
#    print data
    cur.execute(q,data)
    cur.connection.commit()

In [82]:
n=0
for c in df.iterrows():
    if isCommentNew(c[1]):
        print 'New comment, adding'
        addComment(c[1])
    if n==20:sys.exit(1)
    n+=1

New comment, adding
LIKES NULL nan
[u'1798576', -1, u'trumpcoat', '2015-02-01T22:42:00', u'we are capable of meeting all your need in decorative paint with our range of products such as  trump acrylic emulsion  trump texcoat  trump gloss  trump sheen  trump matt  trump silk  protective wood sheen  epoxy  trump flex industrial coat  silkaflex anti-corrosion system for metals and plastics etc. fungi/algae resistant paint & growth & fire/heat retardant paint.be expecting a classic job+234 [0] 80 2828 7950  081 3092 0421trumpcoat@gmail.cominfo@trumpcoat.com.nghttp //trumpcoat.com.ngwww.nairaland.com/trumpcoat  #bbmme pin  7f6a5509', None, 0, 0, True]
New comment, adding
LIKES NULL nan
[u'1798576', -1, u'trumpcoat', '2015-02-12T18:11:00', u'application of  waterproof sealer/vanish waterproof sealer is a varnish final layer applied on exterior paint  texcote  matt  after it is finished and completely dry. it\u2019s used to protect them from dirt  dust stain  weather uv rays  fungai/algae and

SystemExit: 1

To exit: use 'exit', 'quit', or Ctrl-D.


In [53]:
c

(20, parent                  1798828
 nComment                      3
 user                __MISSING__
 time        2014-07-04 12:16:00
 content              i wonder..
 relevant                   None
 nLikes                      NaN
 nShares                     NaN
 Name: 20, dtype: object)

In [54]:
def getCurrentCommentsCount(testComment):
    existingComments=df3[df3['parent']==testComment['parent']]
    return existingComments.shape[0]

In [57]:
c[1]['parent']

u'1798828'

In [58]:
getCurrentCommentsCount(c[1])

14

In [47]:
isCommentNew(df.iloc[1])

parent                                                1798423
nComment                                                    2
user                                              __MISSING__
time                                      2014-07-04 12:20:00
content      loolx  how in gods earth do i highlight a pos...
relevant                                                 None
nLikes                                                    NaN
nShares                                                   NaN
Name: 1, dtype: object


False

In [15]:
testComment=df.head(1)

In [16]:
testComment

,parent,nComment,user,time,content,relevant,nLikes,nShares
0,1798423,1,__MISSING__,2014-07-04 12:17:00,q,None,NaN,NaN


In [18]:
existingComments=df3[df3['parent']==testComment['parent'].values[0]]

In [23]:
matching=existingComments[(existingComments['content']==testComment['content'].values[0])&
                 (existingComments['time']==testComment['time'].values[0])]

In [24]:
matching.shape

(1, 9)

In [48]:
def isCommentNew(testComment):
#    print testComment
    existingComments=df3[df3['parent']==testComment['parent']]
    matching=existingComments[(existingComments['content']==testComment['content'])&
                 (existingComments['time']==testComment['time'])]
    if matching.shape[0]>0:
        return False
    return True